# ChatPromptTemplate과 Model 분리 실행하기

이 노트북에서는 **ChatPromptTemplate**과 **ChatOllama**를 분리하여 각각 실행하는 방법을 알아봅니다.

## 06번 vs 07번 노트북 비교

| 노트북 | 방식 | 코드 |
|--------|------|------|
| **06.chat-prompt** | LCEL 체인 | `chain = template \| model` |
| **07.chat-prompt-model** | 분리 실행 | `template.invoke()` → `model.invoke()` |

## 분리 실행의 장점

1. **중간 결과 확인**: `template.invoke()` 결과를 출력하여 생성된 메시지 검증
2. **디버깅 용이**: 프롬프트 문제와 모델 문제를 분리하여 파악
3. **조건부 처리**: 생성된 프롬프트에 따라 다른 모델 선택 가능
4. **로깅/저장**: 프롬프트와 응답을 별도로 기록

## 실행 흐름

```
┌──────────────┐    template.invoke()    ┌──────────────┐    model.invoke()    ┌──────────────┐
│   입력 변수   │ ─────────────────────▶ │ ChatPrompt   │ ──────────────────▶ │   AI 응답    │
│ {context,    │                         │ Value        │                      │ (AIMessage)  │
│  question}   │                         │ (메시지 리스트)│                      │              │
└──────────────┘                         └──────────────┘                      └──────────────┘
```

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (1,734 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current u

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. ChatPromptTemplate + Model 분리 실행

**코드 구조 설명:**

### Step 1: ChatPromptTemplate 생성
```python
template = ChatPromptTemplate.from_messages([...])
```
- 역할별 메시지 리스트로 템플릿 정의

### Step 2: Model 생성
```python
model = ChatOllama(model='llama3.2')
```
- Chat 모델 인스턴스 생성

### Step 3: template.invoke() - 프롬프트 생성
```python
prompt = template.invoke({'context': '...', 'question': '...'})
```
- 변수를 대입하여 **ChatPromptValue** 객체 생성
- 이 단계에서 중간 결과 확인 가능

### Step 4: model.invoke() - LLM 호출
```python
response = model.invoke(prompt)
```
- 생성된 프롬프트로 모델 호출
- **AIMessage** 객체 반환

### LCEL 체인과의 차이점

```python
# LCEL 체인 (06번 노트북)
chain = template | model
response = chain.invoke({...})  # 한 번에 실행

# 분리 실행 (이 노트북)
prompt = template.invoke({...})  # Step 1: 프롬프트 생성
response = model.invoke(prompt)  # Step 2: 모델 호출
```

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

template = ChatPromptTemplate.from_messages([
    ('system', '''아래 작성한 컨텍스트(Context)를 기반으로
    질문(Question)에 대답하세요. 제공된 정보로 대답할 수 없는 질문이라면 "모르겠어요." 라고 답하세요.'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])

model = ChatOllama(model='llama3.2')  # 여기 수정!

prompt = template.invoke({
    'context': '''거대 언어 모델(LLM)은 자연어 처리(NLP) 분야의 최신 발전을 이끌고 있습니다.
    거대 언어 모델은 더 작은 모델보다 우수한 성능을 보이며, NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게
    매우 중요한 도구가 되었습니다. 개발자들은 Hugging Face의 `transformers` 라이브러리를
    활용하거나, `openai` 및 `cohere` 라이브러리를 통해 OpenAI와 Cohere의 서비스를 이용하여
    거대 언어 모델을 활용할 수 있습니다.
    ''',
    'question': '거대 언어 모델은 어디서 제공하나요?'
})

response = model.invoke(prompt)
print(response.content)

거대 언어 모델은 Hugging Face의 `transformers` 라이브러리, OpenAI의 `dall-e`, `barber` 그리고 `cohere` 서비스를 통해 제공합니다.


---

## 중간 결과 확인하기 (디버깅 팁)

`template.invoke()` 결과를 출력하여 생성된 메시지를 확인할 수 있습니다:

```python
prompt = template.invoke({'context': '...', 'question': '...'})

# 메시지 리스트 확인
for msg in prompt.messages:
    print(f"[{msg.type}]: {msg.content[:50]}...")
```

**출력 예시:**
```
[system]: Context를 기반으로 질문에 대답하세요...
[human]: Context: 거대 언어 모델(LLM)은...
[human]: Question: 거대 언어 모델은 어디서 제공하나요?
```

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai import ChatOpenAI
model = ChatOpenAI()

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 관련 노트북 요약

| 노트북 | Template 타입 | 실행 방식 |
|--------|--------------|----------|
| 04.prompt | PromptTemplate | LCEL 체인 |
| 05.prompt-model | PromptTemplate | 분리 실행 |
| 06.chat-prompt | ChatPromptTemplate | LCEL 체인 |
| **07.chat-prompt-model** | ChatPromptTemplate | **분리 실행** |

## 활용 시나리오

1. **프롬프트 검증**: 복잡한 템플릿이 의도대로 생성되는지 확인
2. **A/B 테스트**: 같은 프롬프트를 여러 모델에 적용하여 비교
3. **프롬프트 캐싱**: 자주 사용하는 프롬프트를 저장했다가 재사용
4. **에러 추적**: 프롬프트 생성 오류와 모델 오류를 분리하여 디버깅